In [1]:
%matplotlib inline

# Diferenciación automática con autograd

Al entrenar redes neuronales, el algoritmo más utilizado es el de **back propagation**. En este algoritmo, los parámetros (pesos del modelo) se ajustan de acuerdo con el **gradiente** de la función de pérdida con respecto al parámetro dado.

Para calcular esos gradientes, PyTorch tiene un motor de diferenciación incorporado llamado ``torch.autograd``. Admite el cálculo automático del gradiente para cualquier gráfico computacional.

Considere la red neuronal de una capa más simple, con entrada ``x``, parámetros ``w`` y ``b``, y alguna función de pérdida. Se puede definir en PyTorch de la siguiente manera:

In [1]:
import torch

In [2]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Tensores, funciones y gráfico computacional

Este código define el siguiente **gráfico computacional**:

![comp-graph](https://pytorch.org/tutorials/_images/comp-graph.png)

En esta red, _w_ y _b_ son parámetros, que necesitamos optimizar. Por lo tanto, necesitamos poder calcular los gradientes de la función de pérdida con respecto a esas variables. Para hacer eso, establecemos la propiedad ``requires_grad`` de esos tensores.

 > **Nota**: Puede establecer el valor de ``requires_grad`` al crear un tensor, o más tarde mediante el método ``x.requires_grad_(True)``.

Una función que aplicamos a los tensores para construir un grafo computacional es de hecho un objeto de clase ``Function``. Este objeto sabe cómo calcular la función en la dirección de avance y también cómo calcular su derivada durante el paso de **propagación hacia atrás**. Una referencia a la función de propagación hacia atrás se almacena en la propiedad ``grad_fn`` de un tensor. Puede encontrar más información de ``Function`` en la [documentación](https://pytorch.org/docs/stable/autograd.html#function).

In [3]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x00000197E0E47D90>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x00000197E0E47CD0>


## Gradientes computacionales

Para optimizar los pesos de los parámetros en la red neuronal, necesitamos calcular las derivadas de nuestra función de pérdida con respecto a los parámetros, es decir, necesitamos $\frac{∂loss}{∂w}$ y $\frac{∂loss}{∂b}$ bajo algunos valores fijos de ``x`` y ``y``. Para calcular esas derivadas, llamamos ``loss.backward()``, y luego recuperamos los valores de ``w.grad`` y ``b.grad``:

In [4]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0179, 0.1184, 0.2114],
        [0.0179, 0.1184, 0.2114],
        [0.0179, 0.1184, 0.2114],
        [0.0179, 0.1184, 0.2114],
        [0.0179, 0.1184, 0.2114]])
tensor([0.0179, 0.1184, 0.2114])


 > **Nota**:
 > * Solo podemos obtener las propiedades grad de los nodos _hoja_ del grafo computacional, que tienen la propiedad ``requires_grad`` establecida en True. Para todos los demás nodos de nuestro gráfico, los gradientes no estarán disponibles.
 > * Solo podemos realizar cálculos de gradiente usando ``backward`` una vez en un gráfico dado, por razones de rendimiento. Si necesitamos hacer varias llamadas ``backward`` en el mismo gráfico, debemos pasar ``retain_graph=True`` a la llamada ``backward``.

## Deshabilitar el seguimiento de gradientes

De forma predeterminada, todos los tensores con ``requires_grad=True`` están rastreando su historial computacional y admiten el cálculo de gradientes. Sin embargo, hay algunos casos en los que no necesitamos hacer eso, por ejemplo, cuando hemos entrenado el modelo y solo queremos aplicarlo a algunos datos de entrada (inferencia), es decir, solo queremos hacer cálculos a través de la red. Podemos detener el seguimiento de los cálculos rodeando nuestro código de cálculo con el bloque ``torch.no_grad()``:

In [5]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


Otra forma de lograr el mismo resultado es usar el método ``detach()`` en el tensor:

In [6]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


Existen motivos por los que quizás desee deshabilitar el seguimiento de gradientes:
 * Para marcar algunos parámetros en su red neuronal como **frozen parameters**. Este es un escenario muy común para ajustar una [finetuning a pretrained network](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)
 * Para acelerar los cálculos cuando solo está haciendo forward, porque los cálculos en tensores que no siguen los gradientes serían más eficientes.

## Más sobre gráficos computacionales

Conceptualmente, autograd mantiene un registro de datos (tensores) y todas las operaciones ejecutadas (junto con los nuevos tensores resultantes) en un gráfico acíclico dirigido (DAG) que consta de objetos de [Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function). En este DAG, las hojas son los tensores de entrada, las raíces son los tensores de salida. Al trazar este gráfico desde las raíces hasta las hojas, puede calcular automáticamente los gradientes usando la regla de la cadena.

En forward, autograd hace dos cosas simultáneamente:
 * Ejecutar la operación solicitada para calcular un tensor resultante
 * Mantener la función de gradiente de la operación en el DAG.


En backward comienza cuando ``.backward()`` se llama en la raíz del DAG. ``autograd`` luego:
 * calcula los gradientes de cada uno ``.grad_fn``
 * los acumula en el atributo ``.grad`` del tensor respectivo
 * utilizando la regla de la cadena, se propaga hasta los tensores de las hojas.

 > **Nota**:
 > **Los DAG son dinámicos en PyTorch**. Una cosa importante a tener en cuenta es que el gráfico se recrea desde cero; después de cada llamada ``.backward()``, autograd comienza a completar un nuevo gráfico. Esto es exactamente lo que le permite utilizar declaraciones de flujo de control en su modelo; puede cambiar la forma, el tamaño y las operaciones en cada iteración si es necesario.

## Lectura opcional: gradientes tensoriales y productos jacobianos

En muchos casos, tenemos una función de pérdida escalar y necesitamos calcular el gradiente con respecto a algunos parámetros. Sin embargo, hay casos en los que la función de salida es un tensor arbitrario. En este caso, PyTorch le permite calcular el llamado producto jacobiano, y no el gradiente real.

Para una función vectorial $\vec{x}=\langle x_1,\dots,x_n\rangle$, dónde $\vec{y}=\langle y_1,\dots,y_m\rangle$ y $\vec{y}=\langle y_1,\dots,y_m\rangle$, un gradiente de $\vec{y}$ con respecto a $\vec{x}$ viene dada por la **matriz jacobiana**:

$$
\begin{align}J=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\end{align}
$$

En lugar de calcular la matriz jacobiana en sí, PyTorch le permite calcular el **producto jacobiano** $v^T\cdot J$ para un vector de entrada dado $v=(v_1 \dots v_m)$. Esto se logra llamando ``backward`` con $v$ como argumento. El tamaño de $v$ debe ser el mismo que el tamaño del tensor original, con respecto al cual queremos calcular el producto:

In [7]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


Observe que cuando llamamos ``backward`` por segunda vez con el mismo argumento, el valor del gradiente es diferente. Esto sucede porque al hacer la propagación ``backward``, PyTorch **acumula los gradientes**, es decir, el valor de los gradientes calculados se suma a la propiedad ``grad`` de todos los nodos hoja del gráfico computacional. Si desea calcular los gradientes adecuados, debe poner a cero la propiedad ``grad`` antes. En el entrenamiento de la vida real, un optimizador nos ayuda a hacer esto.

 > **Nota**:
 > Anteriormente llamábamos a la función ``backward()`` sin parámetros. Esto es esencialmente equivalente a llamar ``backward(torch.tensor(1.0))``, que es una forma útil de calcular los gradientes en el caso de una función con valores escalares, como la pérdida durante el entrenamiento de la red neuronal.